In [6]:
# Introduction to Predictive Coding Models: MNIST Example

# This notebook demonstrates a minimal three-layer predictive coding model for image classification using the MNIST dataset. Each layer predicts the next, computes prediction errors, and updates its states and weights accordingly. Lateral connections are included for richer feature interactions.

# ---

## 1. Import Libraries and Load Data

# ```python
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
# ```

# ---

## 2. Data Preprocessing

# ```python
# Download and load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Use a smaller subset for faster training (for demonstration)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
# ```

# ---

## 3. Predictive Coding Model Definition

### Model Structure

# - **Input Layer:** 784 units (28x28 pixels)
# - **Hidden Layer:** 256 units
# - **Output Layer:** 10 units (digits 0-9)
# - **Lateral Connections:** Within hidden layer

# ```python
class PredictiveCodingModel:
    def __init__(self, input_size=784, hidden_size=256, output_size=10, lr=1e-3):
        # Feedforward weights
        self.W1 = np.random.randn(hidden_size, input_size) * 0.1
        self.W2 = np.random.randn(output_size, hidden_size) * 0.1
        # Lateral weights (hidden layer)
        self.L1 = np.eye(hidden_size) * 0.01  # Small lateral weights
        # States
        self.x1 = np.zeros((hidden_size, 1))
        self.x2 = np.zeros((output_size, 1))
        # Learning rate
        self.lr = lr

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0, keepdims=True)

    def forward(self, x0):
        # x0: input (784, 1)
        # Predict hidden state
        pred_x1 = self.sigmoid(self.W1 @ x0 + self.L1 @ self.x1)
        # Predict output
        pred_x2 = self.softmax(self.W2 @ pred_x1)
        return pred_x1, pred_x2

    def compute_errors(self, x0, x1, x2, y):
        # Prediction errors
        e0 = x0 - self.W1.T @ x1  # Input error
        e1 = x1 - self.sigmoid(self.W1 @ x0 + self.L1 @ x1)  # Hidden error
        e2 = x2 - y  # Output error (y is one-hot)
        return e0, e1, e2

    def update_states(self, x0, x1, x2, y, e0, e1, e2):
        # Update hidden state (x1)
        dx1 = self.W1 @ e0 + self.L1 @ e1 - e1
        x1 += self.lr * dx1
        # Update output state (x2)
        dx2 = -e2
        x2 += self.lr * dx2
        return x1, x2

    def update_weights(self, x0, x1, x2, e0, e1, e2):
        # Update feedforward weights
        self.W1 += self.lr * (np.outer(e1, x0))
        self.W2 += self.lr * (np.outer(e2, x1))
        # Update lateral weights (hidden)
        self.L1 += self.lr * (np.outer(e1, x1))

    def predict(self, x0):
        x1 = self.sigmoid(self.W1 @ x0 + self.L1 @ np.zeros_like(self.x1))
        x2 = self.softmax(self.W2 @ x1)
        return np.argmax(x2)
# ```

# ---

## 4. Training Loop

# ```python
def one_hot(label, num_classes=10):
    vec = np.zeros((num_classes, 1))
    vec[label] = 1
    return vec

model = PredictiveCodingModel()
epochs = 3

for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28).numpy()
        target = target.numpy()
        for i in range(data.shape[0]):
            x0 = data[i].reshape(-1, 1)
            y = one_hot(target[i])
            # Initialize states
            x1 = np.zeros((256, 1))
            x2 = np.zeros((10, 1))
            # Iterative inference (simulate prediction-error minimization)
            for _ in range(5):
                pred_x1, pred_x2 = model.forward(x0)
                e0, e1, e2 = model.compute_errors(x0, x1, x2, y)
                x1, x2 = model.update_states(x0, x1, x2, y, e0, e1, e2)
            # Update weights after inference
            model.update_weights(x0, x1, x2, e0, e1, e2)
            # Loss (sum of squared errors)
            total_loss += np.sum(e2**2)
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {total_loss/(batch_idx+1):.4f}")
# ```

# ---

## 5. Evaluation

# ```python
correct = 0
total = 0
for data, target in test_loader:
    data = data.view(-1, 28*28).numpy()
    target = target.numpy()
    for i in range(data.shape[0]):
        x0 = data[i].reshape(-1, 1)
        pred = model.predict(x0)
        if pred == target[i]:
            correct += 1
        total += 1
print(f"Test Accuracy: {100 * correct / total:.2f}%")
# ```

# ---

## 6. Summary

# This notebook implemented a minimal predictive coding model with three layers and lateral connections for MNIST classification. Each layer predicts the next, computes errors, and updates its states and weights to minimize prediction error. The model demonstrates the core logic of predictive coding and achieves basic classification accuracy.

# ---

# **Note:** This is a minimal, educational implementation. For higher accuracy, use more advanced architectures and optimization techniques.# Predictive Coding Model for MNIST Classification

# --- Cell 1: Install and Import Libraries ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --- Cell 2: Load and Preprocess MNIST Data ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize MNIST
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# --- Cell 3: Define Predictive Coding Layer ---
class PredictiveCodingLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super(PredictiveCodingLayer, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.state = torch.zeros(output_size).to(device)

    def forward(self, input):
        prediction = self.linear(input)
        return prediction

    def update(self, prediction, target, learning_rate=0.01):
        error = target - prediction  # Prediction error
        self.linear.weight.data += learning_rate * torch.outer(error, self.linear.input)
        self.linear.bias.data += learning_rate * error
        return error

# --- Cell 4: Full Predictive Coding Network ---
class PredictiveCodingNet(nn.Module):
    def __init__(self):
        super(PredictiveCodingNet, self).__init__()
        self.l1 = PredictiveCodingLayer(28*28, 256)
        self.l2 = PredictiveCodingLayer(256, 128)
        self.l3 = PredictiveCodingLayer(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        z1 = F.relu(self.l1(x))
        z2 = F.relu(self.l2(z1))
        z3 = self.l3(z2)
        return z1, z2, z3

    def train_predictive_coding(self, x, target, iterations=1, lr=0.01):
        x = x.view(-1, 28*28)
        z1, z2, z3 = self.forward(x)

        # Compute prediction errors layer by layer (simplified)
        error3 = target - z3
        error2 = self.l2.linear.weight.T @ error3 * (z2 > 0)
        error1 = self.l1.linear.weight.T @ error2 * (z1 > 0)

        # Update weights with prediction errors
        self.l3.linear.weight.data += lr * torch.matmul(error3.T, z2)
        self.l2.linear.weight.data += lr * torch.matmul(error2.T, z1)
        self.l1.linear.weight.data += lr * torch.matmul(error1.T, x)

        self.l3.linear.bias.data += lr * error3.sum(0)
        self.l2.linear.bias.data += lr * error2.sum(0)
        self.l1.linear.bias.data += lr * error1.sum(0)

        return error1.norm().item(), error2.norm().item(), error3.norm().item()

# --- Cell 5: Train Model ---
model = PredictiveCodingNet().to(device)
epochs = 5
lr = 0.001

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        target = F.one_hot(labels, num_classes=10).float()
        e1, e2, e3 = model.train_predictive_coding(images, target, lr=lr)
        total_loss += e3  # Track output layer error
    print(f"Epoch {epoch+1} - Output Layer Error: {total_loss:.4f}")

# --- Cell 6: Evaluate Accuracy ---
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            _, _, output = model.forward(images)
            preds = output.argmax(dim=1).cpu()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100.0 * correct / total

acc = evaluate(model, test_loader)
print(f"Test Accuracy: {acc:.2f}%")


Epoch 1, Batch 0, Loss: 63.4898
Epoch 1, Batch 100, Loss: 63.4898
Epoch 1, Batch 200, Loss: 63.4898
Epoch 1, Batch 300, Loss: 63.4898
Epoch 1, Batch 400, Loss: 63.4898
Epoch 1, Batch 500, Loss: 63.4898
Epoch 1, Batch 600, Loss: 63.4898
Epoch 1, Batch 700, Loss: 63.4898
Epoch 1, Batch 800, Loss: 63.4898
Epoch 1, Batch 900, Loss: 63.4898
Epoch 2, Batch 0, Loss: 63.4898
Epoch 2, Batch 100, Loss: 63.4898
Epoch 2, Batch 200, Loss: 63.4898
Epoch 2, Batch 300, Loss: 63.4898
Epoch 2, Batch 400, Loss: 63.4898
Epoch 2, Batch 500, Loss: 63.4898
Epoch 2, Batch 600, Loss: 63.4898
Epoch 2, Batch 700, Loss: 63.4898
Epoch 2, Batch 800, Loss: 63.4898
Epoch 2, Batch 900, Loss: 63.4898
Epoch 3, Batch 0, Loss: 63.4898
Epoch 3, Batch 100, Loss: 63.4898
Epoch 3, Batch 200, Loss: 63.4898
Epoch 3, Batch 300, Loss: 63.4898
Epoch 3, Batch 400, Loss: 63.4898
Epoch 3, Batch 500, Loss: 63.4898
Epoch 3, Batch 600, Loss: 63.4898
Epoch 3, Batch 700, Loss: 63.4898
Epoch 3, Batch 800, Loss: 63.4898
Epoch 3, Batch 900, 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x128 and 64x10)